In [15]:
import pandas as pd

df = pd.read_csv('/kaggle/input/fraud-dataset/Fraud.csv')
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [16]:
df.isnull().any()

step              False
type              False
amount            False
nameOrig          False
oldbalanceOrg     False
newbalanceOrig    False
nameDest          False
oldbalanceDest    False
newbalanceDest    False
isFraud           False
isFlaggedFraud    False
dtype: bool

In [17]:
df.dtypes

step                int64
type               object
amount            float64
nameOrig           object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest           object
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
dtype: object

In [18]:
impute_cols = []
for col in df.select_dtypes(['int', 'float']):
    if col == 'isFraud' or col == 'isFlaggedFraud': continue
    n = df[df[col] == 0].shape[0]
    if n > 0  :
        impute_cols.append(col)
impute_cols

['amount',
 'oldbalanceOrg',
 'newbalanceOrig',
 'oldbalanceDest',
 'newbalanceDest']

In [19]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import numpy as np

impute_df = df[impute_cols]
impute_df.replace(0, np.nan, inplace=True)
imp = IterativeImputer(max_iter=10, random_state=32, min_value=0)
imputed_data = imp.fit_transform(impute_df)

/tmp/ipykernel_30/2318344910.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  impute_df.replace(0, np.nan, inplace=True)
/opt/conda/lib/python3.10/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [20]:
imputed_df = pd.DataFrame(imputed_data, columns=impute_df.columns)
imputed_df = imputed_df.clip(lower = 0)
for col in imputed_df.columns:
    print(col, imputed_df[imputed_df[col] == 0].shape[0])

amount 0
oldbalanceOrg 10781
newbalanceOrig 115044
oldbalanceDest 210178
newbalanceDest 108635


In [21]:
for col in imputed_df.columns:
    df[col] = imputed_df[col]

for col in df.columns:
    n = df[df[col] == 0.0].shape[0]
    if (col  in impute_cols) & (n >= 0):
        df[df[col] == 0.0][col] = np.mean(df[col])

/tmp/ipykernel_30/1639275705.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df[col] == 0.0][col] = np.mean(df[col])


In [22]:
df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798623e+05,1.241033e+06,1.318871e+06,1.701193e+06,1.822743e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.830422e+06,2.856581e+06,3.298340e+06,3.570307e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,1.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338980e+04,2.559800e+04,7.339868e+04,3.820531e+05,4.888375e+05,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487349e+04,2.719947e+05,3.929842e+05,1.442561e+06,1.536047e+06,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087216e+05,1.284833e+06,1.343395e+06,1.737143e+06,1.775993e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


In [23]:
df.drop(columns=['nameOrig', 'nameDest'], inplace=True)

df_encoded = pd.get_dummies(data=df, columns=['type'])
df_encoded[df_encoded.select_dtypes('bool').columns] = df_encoded[df_encoded.select_dtypes('bool').columns].astype(int)

In [24]:
corr_matrix = df_encoded.corr()

In [25]:
corr_matrix

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
step,1.000000,0.022374,-0.012005,-0.009694,0.030008,0.028777,0.031578,0.003277,0.004077,-0.012918,0.002869,0.004926,0.006925
amount,0.022374,1.000000,-0.012153,0.056439,0.261318,0.437916,0.076706,0.012295,-0.009621,-0.004375,-0.023379,-0.197444,0.365896
oldbalanceOrg,-0.012005,-0.012153,1.000000,0.996820,0.034513,0.015473,0.005260,0.003685,0.440827,-0.164771,-0.028041,-0.174105,-0.069635
newbalanceOrig,-0.009694,0.056439,0.996820,1.000000,0.054030,0.042561,0.007402,0.003608,0.453628,-0.172062,-0.030084,-0.191437,-0.046016
oldbalanceDest,0.030008,0.261318,0.034513,0.054030,1.000000,0.973072,-0.003462,0.000121,0.014479,-0.044576,-0.005107,-0.014584,0.081572
newbalanceDest,0.028777,0.437916,0.015473,0.042561,0.973072,1.000000,0.020086,0.004985,-0.026645,-0.026668,-0.007007,-0.036444,0.150078
isFraud,0.031578,0.076706,0.005260,0.007402,-0.003462,0.020086,1.000000,0.044109,-0.019089,0.011256,-0.002911,-0.025697,0.053869
isFlaggedFraud,0.003277,0.012295,0.003685,0.003608,0.000121,0.004985,0.044109,1.000000,-0.000842,-0.001168,-0.000128,-0.001133,0.005245
type_CASH_IN,0.004077,-0.009621,0.440827,0.453628,0.014479,-0.026645,-0.019089,-0.000842,1.000000,-0.391047,-0.042987,-0.379522,-0.160535
type_CASH_OUT,-0.012918,-0.004375,-0.164771,-0.172062,-0.044576,-0.026668,0.011256,-0.001168,-0.391047,1.000000,-0.059625,-0.526422,-0.222672


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = pd.DataFrame()
label_cols = [col for col in df_encoded.columns if col != 'isFraud']
vif_data['features'] = label_cols
vif_data["VIF"] = [variance_inflation_factor(df_encoded[label_cols].values, i) for i in range(df_encoded[label_cols].shape[1])]
print(vif_data)

In [27]:
df_encoded['old_balance_diff'] = df_encoded['newbalanceOrig'] - df_encoded['oldbalanceOrg']
df_encoded['new_balance_diff'] = df_encoded['newbalanceDest'] - df_encoded['oldbalanceDest']
df_encoded.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,old_balance_diff,new_balance_diff
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798623e+05,1.241033e+06,1.318871e+06,1.701193e+06,1.822743e+06,1.290820e-03,2.514687e-06,2.199226e-01,3.516633e-01,6.511783e-03,3.381461e-01,8.375622e-02,7.783822e+04,1.215504e+05
std,1.423320e+02,6.038582e+05,2.830422e+06,2.856581e+06,3.298340e+06,3.570307e+06,3.590480e-02,1.585775e-03,4.141940e-01,4.774895e-01,8.043246e-02,4.730786e-01,2.770219e-01,2.282626e+05,8.415299e+05
min,1.000000e+00,1.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.034601e+07,-5.353304e+06
25%,1.560000e+02,1.338980e+04,2.559800e+04,7.339868e+04,3.820531e+05,4.888375e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.075075e+02,-5.394269e+04
50%,2.390000e+02,7.487349e+04,2.719947e+05,3.929842e+05,1.442561e+06,1.536047e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.865966e+04,2.761297e+04
75%,3.350000e+02,2.087216e+05,1.284833e+06,1.343395e+06,1.737143e+06,1.775993e+06,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.032328e+05,1.568899e+05
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.309645e+07,1.056878e+08


In [ ]:
df_encoded.columns

In [28]:
df_final = df_encoded.copy()
X = df_final.drop(columns=['isFraud'])
y = df_final.loc[:, 'isFraud']
X.shape, y.shape

((6362620, 14), (6362620,))

In [29]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=32)
for tr_idx, test_idx in sss.split(X, y):
    X_train, y_train = X.iloc[tr_idx], y.iloc[tr_idx]
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((3817572, 14), (3817572,), (2545048, 14), (2545048,))

In [ ]:
# import optuna
# from sklearn.model_selection import cross_val_score, cross_validate
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import PolynomialFeatures, StandardScaler
# from sklearn.metrics import make_scorer, fbeta_score, f1_score, precision_score, recall_score, roc_auc_score
# import xgboost as xgb
# from xgboost import XGBClassifier
# import numpy as np

# sss2 = StratifiedShuffleSplit(n_splits=5, test_size=0.4, random_state=32)
# def objective(trial:optuna.Trial):

#     params = {
#         'learning_rate':trial.suggest_float('learning_rate', 0.1, 0.5, log=True),
#         'n_estimators':trial.suggest_int('n_estimators', 10, 1000),
#         'gamma':trial.suggest_int('gamma', 0, 5),
#         'subsample':trial.suggest_float('subsample', 0.5, 1.0),
#         'lambda':trial.suggest_float('lambda', 0.0, 1.0),
#         'alpha':trial.suggest_float('alpha', 0.0, 1.0),
#     }
    
#     model = XGBClassifier(random_state=42, booster='gbtree', device='cuda', **params)
#     pipe = Pipeline([
#         # ('poly', PolynomialFeatures(interaction_only=True, include_bias=False, degree=2)),
#         ('scaler', StandardScaler()),
#         ('model', model)
#     ])
#     scoring = {
#         'f_3_score':make_scorer(fbeta_score, beta=3),
#         'precision_score':make_scorer(precision_score),
#         'recall_score':make_scorer(recall_score)
#     }
#     scores = cross_validate(pipe, X_train, y_train, cv=sss2, scoring=scoring)
#     f_3_score_mean = np.mean(scores['test_f_3_score'])
#     precision_score_mean = np.mean(scores['test_precision_score'])
#     recall_score_mean = np.mean(scores['test_recall_score'])
#     return (f_3_score_mean, precision_score_mean, recall_score_mean)



In [ ]:
# class EarlyStoppingCallback:
#     def __init__(self, patience:int=10):
#         self.patience = patience
#         self.best_f3 = 0
#         self.counter = 0
#     def __call__(self, study, trial):
#         (f3, pre, recall) = trial.values
#         if self.best_f3 < f3: 
#             self.best_f3 = f3
#             self.counter = self.counter + 1
#         if self.patience == self.counter:
#             print('stopping the study')
#             study.stop()
        
            

In [ ]:
# !rm /kaggle/working/study_1_xgb.db

In [ ]:

# study = optuna.create_study(
#     directions=['maximize', 'maximize', 'maximize'],
#     study_name='study_1_xgb',
#     storage='sqlite:///study_1_xgb.db'
# )
# earlystopping = EarlyStoppingCallback(patience=25)
# study.optimize(objective, n_trials=100, callbacks=[earlystopping], n_jobs=-1, show_progress_bar=True)

In [30]:
import optuna
final_study = optuna.load_study(storage = 'sqlite:////kaggle/input/fraud-optuna-studies/study_1_xgb (1).db', study_name='study_1_xgb')

In [31]:
for trial in final_study.best_trials:
    best_params = trial.params
    print(f'values: {trial.values}')
    break

best_params

values: [0.8290878175959886, 0.9647442748978744, 0.8163368848300354]


{'learning_rate': 0.11823271771432628,
 'n_estimators': 407,
 'gamma': 2,
 'subsample': 0.6947975377148357,
 'lambda': 0.5419321307212279,
 'alpha': 0.9567607906625019}

In [32]:
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

final_model = XGBClassifier(random_state=42, verbosity=2, booster='gbtree', device='cuda', **best_params)
print(f'fitting the train data with the best hyperparams found from optuna.')
final_model.fit(X_train_scaled, y_train)

fitting the train data with the best hyperparams found from hyperopt.


XGBClassifier(alpha=0.9567607906625019, base_score=None, booster='gbtree',
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=2, grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=0.5419321307212279,
              learning_rate=0.11823271771432628, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=407, n_jobs=None, ...)

In [33]:
from sklearn.metrics import make_scorer, roc_auc_score, f1_score, r2_score, precision_score, recall_score, fbeta_score
import pandas as pd

print(f'Calculating the scores on the test set: ')
X_test_scaled = scaler.transform(X_test)
y_pred = final_model.predict(X_test_scaled)

columns = ['f1', 'roc_auc',  'precision', 'recall', 'f_2_score', 'f_3_score']
scores = [f1_score(y_test, y_pred), roc_auc_score(y_test, y_pred),  precision_score(y_test, y_pred), recall_score(y_test, y_pred), fbeta_score(y_test, y_pred, beta=2), fbeta_score(y_test, y_pred, beta=3)]
res_df = pd.DataFrame([scores], columns=columns)
res_df.head()

Calculating the scores on the test set: 


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [18:09:13] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


,f1,roc_auc,precision,recall,f_2_score,f_3_score
0,0.890671,0.914135,0.963186,0.828311,0.852177,0.840074
